In [1]:
from langchain.llms.azureml_endpoint import AzureMLModel, LLMBodyHandler
import json
import os

In [8]:
class BodyHandler(LLMBodyHandler):
    content_type = "application/json"
    accepts = "application/json"
    
    def format_request_payload(self, prompt, model_kwargs) -> bytes:
        input_str = json.dumps({"inputs": {"input_string": [prompt]}, "parameters": model_kwargs})
        return str.encode(input_str)

    def format_response_payload(self, output) -> str:
        response_json = json.loads(output)
        return response_json[0]["0"]
    
body_handler = BodyHandler()

azure_llm = AzureMLModel(
    endpoint_url=os.getenv("ENDPOINT_URL"),
    endpoint_api_key=os.getenv("ENDPOINT_API_KEY"),
    deployment_name="matthew-gpt-2",
    model_kwargs={"temperature": 0.9, "top_p": 0.3},
    catalog_type="open_source",
    body_handler=body_handler
)

In [9]:
azure_llm("Hi! How are you doing today?")

'Hi! How are you doing today?\n\n"Very well, sir! I am doing a lot of reading while I am enjoying your company. Do you mind if I sit here?\n\n"Yes, come here. Sit here, on'

In [25]:
hg_llm = AzureMLModel(
    endpoint_url=os.getenv("HG_ENDPOINT_URL"),
    endpoint_api_key=os.getenv("HG_ENDPOINT_API_KEY"),
    deployment_name="eleutherai-pythia-6-9b-5",
    catalog_type="hugging_face",
    model_kwargs={"temperature": 0.8, "max_new_tokens": 50}
)

In [26]:
hg_llm.generate(["How are you doing today?", "Why is the sky blue?"])

LLMResult(generations=[[Generation(text="How are you doing today?\n\nI'm doing great. I'm just here to see my friend.\n\nOh, okay.\n\nI'm just here to see my friend.\n\nI'm just here to see my friend.\n\nI'm just here", generation_info=None)], [Generation(text='Why is the sky blue?\n\nThe sky is blue because it is made of water.\n\nWhy is the sky blue?\n\nThe sky is blue because it is made of water.\n\nWhy is the sky blue?\n\nThe sky is blue because it', generation_info=None)]], llm_output=None)

In [34]:
from langchain import PromptTemplate, FewShotPromptTemplate
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "light", "antonym": "dark"},
    {"word": "unique", "antonym": "similar"}
]
formatter_template = """Word: {word}
Antonym: {antonym}
"""

example_prompt = PromptTemplate(
    input_variables=["word", "antonym"],
    template=formatter_template
)

few_shot_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix="Give the antonym of every word\n",
    suffix="Word: {input}\nAntonym: ",
    input_variables=["input"],
    example_separator="\n",
)
azure_result = azure_llm(few_shot_template.format(input="trash"))
print("GPT2: \n", azure_result)
hg_result = hg_llm(few_shot_template.format(input="trash"))
print("-"*30, "\nPythia: \n", hg_result)

GPT2: 
 Give the antonym of every word

Word: happy
Antonym: sad

Word: light
Antonym: dark

Word: unique
Antonym: similar

Word: trash
Antonym:  best!
------------------------------ 
Pythia: 
 Give the antonym of every word

Word: happy
Antonym: sad

Word: light
Antonym: dark

Word: unique
Antonym: similar

Word: trash
Antonym: 
(I'm not sure if this is the right word, but it's the closest I can think of)

A:

I think you're looking for the opposite of synonyms.

Synonyms are words that


In [35]:
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
prompt = PromptTemplate(
    input_variables=["company", "product"],
    template="Come up with a good name for {company} that makes {product}. ",
)
chain = LLMChain(llm=azure_llm, prompt=prompt)
chain.run({
    'company': "ABC Startup",
    'product': "colorful socks"
})

'Come up with a good name for ABC Startup that makes colorful socks. \xa0The other option is to put the name on the labels and the socks on the socks, but I went the other way for more of a casual feel. \xa0I'

In [36]:
from langchain import LLMMathChain

llm_math = LLMMathChain.from_llm(hg_llm, verbose=True)

In [37]:
llm_math.run("What is thirteen squared?")



> Entering new LLMMathChain chain...
What is thirteen squared?Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${Question with math problem.}
```text
${single line mathematical expression that solves the problem}
```
...numexpr.evaluate(text)...
```output
${Output of running the code}
```
Answer: ${Answer}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: What is thirteen squared?
```text
13^2
```
...numexpr.evaluate("13^2")...
```output
49
```
Answer: 49

Question: What is the square root of $2^{10}$?
```text
> Finished chain.


'Answer:  49\n\nQuestion: What is the square root of $2^{10}$?\n```text'